In [21]:
import math
import random

first_frame = 4520 # first frame that the blower leaves
end_first_acceleration = 4695 # at this point, both the blower and the coyote move at the same speed

bursts = (4740, 4750, 4765, 4784, 4805, 4845, 4868, 4879, 4906, 4918, 4929, 4936, 4941, 4948, 4952, 4958) # at these points, the blower shoots out a ball to accelerate further

last_frame = 4983 # last frame just before blower hits wall
catchup_frame = 5268 # at this point, wile stops and seems to be pretty close to the blower

height_body = 1.295 # height of blower in meters
radius_body = 0.94 / 2 # radius of blower in meters

height_top = 0.147 # height of top of blower in meters
radius_top = 0.474 / 2 # radius of top of blower in meters

total_time = (last_frame - first_frame) / 30 # total time in seconds
max_speed = 43 / 2.237 # max speed of animal converted from mph to m/s
normal_acceleration = max_speed / ((end_first_acceleration - first_frame) / 30) # acceleration in m/s^2

In [22]:
catchup_time = (catchup_frame - last_frame) / 30 # amount of time it takes of wile to catch up with blower
distance_ahead = max_speed * catchup_time # distance that the blower was ahead just before it hit the wall

blower_ahead_average_speed = distance_ahead / ((last_frame - bursts[0]) / 30) # the average speed of the time that the blower moves faster than the coyote. calculated by when the blower first starts moving faster (at the first burst) until it hits the wall

burst_speed_increase = blower_ahead_average_speed / len(bursts) # the amount of speed gained by each burst, assuming each burst is the same size

In [23]:
volume_body = math.pi * (radius_body**2) * height_body # volume of blower in m^3
volume_air = math.pi * ((radius_body - 0.04)**2) * (height_body - 0.04) # volume of air in m^3

volume_top = math.pi * (radius_top**2) * height_top # volume of top of blower in m^3

volume_aluminium = volume_body - volume_air + volume_top # volume of aluminium in m^3

density_aluminium = 2700 # density of aluminium in kg/m^3
mass_aluminium = volume_aluminium * density_aluminium # mass of aluminium in kg

density_air = 1.293 # density of air in kg/m^3
mass_air = volume_air * density_air # mass of air in kg

In [24]:
weight = mass_aluminium + mass_air # weight of blower in kg
normal_force = 9.81 * weight

roll_friction_coefficient = 0.01 # friction coefficient of the ground with the wheels

ground_friction_newton = roll_friction_coefficient * normal_force

frontal_area = 2 * math.pi * radius_body * height_body / 2 # frontal area of blower in m^2

max_air_friction_newton = 0.5 * density_air * (max_speed ** 2) * 0.47 * frontal_area # air friction in newton at max speed
total_power_newton = ground_friction_newton + max_air_friction_newton # total friction in newton at max speed

In [25]:
from plotly.offline import init_notebook_mode
import plotly.graph_objects as go

init_notebook_mode(connected=True)

total_frames = catchup_frame - first_frame # total frames. framerate is 30fps
delta_t = 1/30

acceleration = 0
velocity = 0
distance = 0

time_data = []
acceleration_data = []
velocity_data = []
distance_data = []

def get_friction(current_speed: float) -> float:
    air_friction_newton = 0.5 * density_air * (current_speed ** 2) * 0.47 * frontal_area # air friction in newton
    total_friction_newton = ground_friction_newton + air_friction_newton # total friction in newton
    
    return (total_power_newton - total_friction_newton) / weight # netto F gedeeld door m is versnelling

def get_acceleration(current_frame: int) -> float:
    delta_acceleration = 0
    if first_frame <= current_frame <= end_first_acceleration:
        delta_acceleration += normal_acceleration

    if current_frame in bursts:
        delta_acceleration += burst_speed_increase

    return delta_acceleration


for i in range(total_frames):
    frame = i + first_frame
    
    acceleration = get_acceleration(frame) - get_friction(velocity)
    velocity += acceleration * delta_t
    distance += velocity * delta_t
    
    acceleration_data.append(acceleration)
    velocity_data.append(velocity)
    distance_data.append(distance)
    time_data.append(frame * delta_t)

acceleration_graph = go.Scatter(
    x=time_data,
    y=acceleration_data,
    name="Versnelling stofzuiger"
)

velocity_graph = go.Scatter(
    x=time_data,
    y=velocity_data,
    name="Snelhijd stofzuiger"
)    
    
distance_graph = go.Scatter(
    x=time_data,
    y=distance_data,
    name="Afstand stofzuiger"
)

acceleration_fig = go.Figure(acceleration_graph, layout=dict(
    title = "Versnelling van de stofzuiger",
    xaxis_title="Tijd (s)",
    yaxis_title="Versnelling (m/s^2)",
    autosize=False
))

velocity_fig = go.Figure(velocity_graph, layout=dict(
    title = "Snelheid van de stofzuiger",
    xaxis_title="Tijd (s)",
    yaxis_title="Snelheid (m/s)",
    autosize=False
))

distance_fig = go.Figure(distance_graph, layout=dict(
    title = "Afstand van de stofzuiger",
    xaxis_title="Tijd (s)",
    yaxis_title="Afstand (m)",
    autosize=False
))

acceleration_fig.show()
velocity_fig.show()
distance_fig.show()